In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2
from fastai.structured import *
from fastai.column_data import *
np.set_printoptions(threshold=50, edgeitems=20)
import pandas as pd
import numpy as np
PATH = '/Users/orendar/Downloads/dataverse_files/'

In [2]:
train_df = pd.read_csv(f'{PATH}train_raw.csv', index_col = 0)
#train_df.head()

,year,camp.length,deminc,base.poll,base.und,office,d.gone.neg,d.gone.neg.l1,d.gone.neg.l2,d.neg.frac.l3,...,rep.contrib.l1,race,first.week,num.dem,demprcnt,d.neg.rec,rep.contrib.l2,d.neg.dur,dem.polls.l2,pos.prob.subset
1,2002,23,1,45.744681,6.0,0,0,0,0,0.000000,...,132545.00,AL-1-2002,-22,303,49.883698,0,102559.36,0,45.744681,True
2,2002,23,1,45.744681,6.0,0,0,0,0,0.000000,...,522327.00,AL-1-2002,-22,232,49.883698,0,132545.00,0,45.744681,True
3,2002,23,1,45.744681,6.0,0,1,1,0,0.000000,...,177739.44,AL-1-2002,-22,805,49.883698,2,138533.00,2,45.744681,True
4,2002,23,1,45.744681,6.0,0,1,1,1,0.000000,...,543465.00,AL-1-2002,-22,748,49.883698,3,177739.44,3,48.536252,True
5,2002,23,1,45.744681,6.0,0,1,1,1,0.071429,...,2415314.50,AL-1-2002,-22,830,49.883698,4,543465.00,4,48.863636,True


In [3]:
test_df = pd.read_csv(f'{PATH}test_raw.csv',  index_col = 0)
#test_df.head()

,year,camp.length,deminc,base.poll,base.und,office,d.gone.neg,d.gone.neg.l1,d.gone.neg.l2,d.neg.frac.l3,...,rep.contrib.l1,race,first.week,num.dem,demprcnt,d.neg.rec,rep.contrib.l2,d.neg.dur,dem.polls.l2,pos.prob.subset
1,2002,13,0,44.736842,24.0,1,1,1,1,1.000000,...,289158.00,CO-2-2002,-12,443,47.446492,5,344029.00,6,48.648649,True
2,2006,23,1,63.636364,12.0,0,0,0,0,0.000000,...,34550.00,NM-1-2006,-22,374,68.817354,0,8641.00,0,70.114943,True
3,2002,23,0,62.765957,6.0,0,1,1,1,0.176471,...,31510.13,NM-1-2002,-22,362,58.696455,5,390706.48,6,60.674157,True
4,2006,27,1,62.637363,9.0,1,0,0,0,0.090909,...,219018.00,WV-2-2006,-26,160,65.646693,0,194750.00,2,67.741935,True
5,2006,15,1,66.455237,9.0,0,0,0,0,0.000000,...,91269.03,TN-1-2006,-14,642,69.756769,0,66217.39,0,66.455237,True


In [4]:
cat_features = ["deminc", "d.gone.neg.l1", "d.gone.neg.l2", "week", "year",
                "office"]

cont_features = ["base.poll", "r.neg.frac.l2", "r.neg.frac.l3", "rep.contrib.l1", 
                 "rep.contrib.l2", "dem.contrib.l1", "dem.contrib.l2", "num.rep.l1",
                 "num.rep.l2", "num.dem.l1", "num.dem.l2", "d.neg.frac.l3", "undother.l2",
                 "undother.l1", "camp.length", "base.und", "dem.polls.l1", "dem.polls.l2",
                "neg.rep.l1", "neg.rep.l2"]

target = "d.gone.neg"

In [5]:
train_df = train_df[cat_features+cont_features+[target]]
x_train, y_train, nas, mapper = proc_df(train_df, target, do_scale=True)

test_df = test_df[cat_features+cont_features+[target]]
x_test, y_test, nas, mapper = proc_df(test_df, target, do_scale=True, mapper=mapper, na_dict=nas)

x_train.shape, y_train.shape, x_test.shape, y_test.shape

((863, 40), (863,), (287, 40), (287,))

In [6]:
from sklearn.ensemble import *
from sklearn.metrics import *
rs = 42

In [8]:
gbc = GradientBoostingClassifier(n_estimators=100, max_features='sqrt', random_state=rs)
rfc = RandomForestClassifier(n_estimators=500, random_state=rs, n_jobs=-1)
etc = ExtraTreesClassifier(n_estimators=500, random_state=rs, n_jobs=-1)

In [9]:
gbc.fit(x_train, y_train)
rfc.fit(x_train, y_train)
etc.fit(x_train, y_train);

In [10]:
gbc_preds = gbc.predict(x_test)
rfc_preds = rfc.predict(x_test)
etc_preds = etc.predict(x_test)
gbc_prob_preds = gbc.predict_proba(x_test)
rfc_prob_preds = rfc.predict_proba(x_test)
etc_prob_preds = etc.predict_proba(x_test)
gbc_pos = gbc_prob_preds.T[1]
rfc_pos = rfc_prob_preds.T[1]
etc_pos = etc_prob_preds.T[1]

In [11]:
def rmse(preds): return mean_squared_error(y_test, preds)**.5
def auroc(preds): return roc_auc_score(y_test, preds)
def accuracy(preds): return accuracy_score(y_test, preds)
def logloss(preds): return log_loss(y_test, preds)

In [12]:
rmse(gbc_pos), rmse(rfc_pos), rmse(etc_pos)

(0.288994977945639, 0.29558670176081964, 0.2991263469314788)

In [13]:
accuracy(gbc_preds), accuracy(rfc_preds), accuracy(etc_preds)

(0.8954703832752613, 0.8780487804878049, 0.867595818815331)

In [14]:
auroc(gbc_pos), auroc(rfc_pos), auroc(etc_pos)

(0.9440683430045133, 0.9414893617021276, 0.9408446163765312)

In [15]:
logloss(gbc_pos), logloss(rfc_pos), logloss(etc_pos)

(0.2802019953829483, 0.2960417907871699, 0.29041988818556763)

In [16]:
df = pd.DataFrame(data=[[accuracy(gbc_preds), rmse(gbc_pos), auroc(gbc_pos), logloss(gbc_pos)],
                        [accuracy(rfc_preds), rmse(rfc_pos), auroc(rfc_pos), logloss(rfc_pos)],
                        [accuracy(etc_preds), rmse(etc_pos), auroc(etc_pos), logloss(etc_pos)]],
                  index=['GBM', 'RF', 'ET'],
                  columns=['Accuracy', 'RMSE', 'AUROC', 'LogLoss'])
df.head()

,Accuracy,RMSE,AUROC,LogLoss
GBM,0.895470,0.288995,0.944068,0.280202
RF,0.878049,0.295587,0.941489,0.296042
ET,0.867596,0.299126,0.940845,0.290420


In [17]:
df.to_latex()

'\\begin{tabular}{lrrrr}\n\\toprule\n{} &  Accuracy &      RMSE &     AUROC &   LogLoss \\\\\n\\midrule\nGBM &  0.895470 &  0.288995 &  0.944068 &  0.280202 \\\\\nRF  &  0.878049 &  0.295587 &  0.941489 &  0.296042 \\\\\nET  &  0.867596 &  0.299126 &  0.940845 &  0.290420 \\\\\n\\bottomrule\n\\end{tabular}\n'

In [18]:
from keras.models import *
from keras.layers import *
from keras.utils import *
from keras.optimizers import *
from sklearn.preprocessing import *
from clr import *
np.random.seed(42)

/Users/orendar/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [21]:
scaler = StandardScaler()
x_train_norm = scaler.fit_transform(x_train)
x_test_norm = scaler.transform(x_test)

In [22]:
model = Sequential()
model.add(Dense(4096, input_dim=40, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Compile model
opt = optimizers.adam(amsgrad=True)
#opt = optimizers.SGD(momentum=0.9, nesterov=True)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['mse', 'accuracy'])

#callbacks arsenal
def sched(idx, lr): return lr*0.9 if idx % 10 == 0 else lr
lrsched = LearningRateScheduler(sched, verbose=0)
reducelr = ReduceLROnPlateau(factor=0.8, patience=2)
stop = EarlyStopping(patience=50)
clr = CyclicLR(base_lr=1e-5, max_lr=1e-4, step_size=15, mode='triangular', scale_mode='cycle')

In [23]:
# Fit the model
model.fit(x_train_norm, y_train, validation_data=(x_test_norm, y_test), epochs=3000, batch_size=512,
          callbacks=[clr, stop], verbose=2)

# evaluate the model
scores = model.evaluate(x_test_norm, y_test)
model.metrics_names, scores

Train on 863 samples, validate on 287 samples
Epoch 1/3000
 - 0s - loss: 0.7162 - mean_squared_error: 0.2614 - acc: 0.3801 - val_loss: 0.7130 - val_mean_squared_error: 0.2599 - val_acc: 0.3693
Epoch 2/3000
 - 0s - loss: 0.7122 - mean_squared_error: 0.2594 - acc: 0.4114 - val_loss: 0.7035 - val_mean_squared_error: 0.2552 - val_acc: 0.4216
Epoch 3/3000
 - 0s - loss: 0.7011 - mean_squared_error: 0.2539 - acc: 0.4531 - val_loss: 0.6903 - val_mean_squared_error: 0.2486 - val_acc: 0.5192
Epoch 4/3000
 - 0s - loss: 0.6897 - mean_squared_error: 0.2483 - acc: 0.5110 - val_loss: 0.6734 - val_mean_squared_error: 0.2402 - val_acc: 0.6794
Epoch 5/3000
 - 0s - loss: 0.6715 - mean_squared_error: 0.2393 - acc: 0.6188 - val_loss: 0.6535 - val_mean_squared_error: 0.2303 - val_acc: 0.7700
Epoch 6/3000
 - 0s - loss: 0.6464 - mean_squared_error: 0.2270 - acc: 0.7358 - val_loss: 0.6312 - val_mean_squared_error: 0.2194 - val_acc: 0.7979
Epoch 7/3000
 - 0s - loss: 0.6300 - mean_squared_error: 0.2189 - acc: 0.

Epoch 57/3000
 - 0s - loss: 0.3887 - mean_squared_error: 0.1178 - acc: 0.8528 - val_loss: 0.3762 - val_mean_squared_error: 0.1134 - val_acc: 0.8502
Epoch 58/3000
 - 0s - loss: 0.3873 - mean_squared_error: 0.1175 - acc: 0.8656 - val_loss: 0.3750 - val_mean_squared_error: 0.1130 - val_acc: 0.8502
Epoch 59/3000
 - 0s - loss: 0.3865 - mean_squared_error: 0.1172 - acc: 0.8575 - val_loss: 0.3742 - val_mean_squared_error: 0.1127 - val_acc: 0.8502
Epoch 60/3000
 - 0s - loss: 0.3870 - mean_squared_error: 0.1178 - acc: 0.8517 - val_loss: 0.3737 - val_mean_squared_error: 0.1125 - val_acc: 0.8502
Epoch 61/3000
 - 0s - loss: 0.3886 - mean_squared_error: 0.1183 - acc: 0.8528 - val_loss: 0.3734 - val_mean_squared_error: 0.1124 - val_acc: 0.8502
Epoch 62/3000
 - 0s - loss: 0.3866 - mean_squared_error: 0.1173 - acc: 0.8494 - val_loss: 0.3727 - val_mean_squared_error: 0.1122 - val_acc: 0.8502
Epoch 63/3000
 - 0s - loss: 0.3836 - mean_squared_error: 0.1165 - acc: 0.8505 - val_loss: 0.3718 - val_mean_squa

Epoch 113/3000
 - 0s - loss: 0.3306 - mean_squared_error: 0.0984 - acc: 0.8795 - val_loss: 0.3239 - val_mean_squared_error: 0.0965 - val_acc: 0.8746
Epoch 114/3000
 - 0s - loss: 0.3275 - mean_squared_error: 0.0974 - acc: 0.8806 - val_loss: 0.3229 - val_mean_squared_error: 0.0963 - val_acc: 0.8746
Epoch 115/3000
 - 0s - loss: 0.3296 - mean_squared_error: 0.0983 - acc: 0.8783 - val_loss: 0.3220 - val_mean_squared_error: 0.0960 - val_acc: 0.8746
Epoch 116/3000
 - 0s - loss: 0.3267 - mean_squared_error: 0.0973 - acc: 0.8749 - val_loss: 0.3212 - val_mean_squared_error: 0.0958 - val_acc: 0.8746
Epoch 117/3000
 - 0s - loss: 0.3267 - mean_squared_error: 0.0974 - acc: 0.8795 - val_loss: 0.3207 - val_mean_squared_error: 0.0957 - val_acc: 0.8746
Epoch 118/3000
 - 0s - loss: 0.3246 - mean_squared_error: 0.0968 - acc: 0.8749 - val_loss: 0.3202 - val_mean_squared_error: 0.0955 - val_acc: 0.8746
Epoch 119/3000
 - 0s - loss: 0.3239 - mean_squared_error: 0.0965 - acc: 0.8818 - val_loss: 0.3199 - val_me

Epoch 168/3000
 - 0s - loss: 0.2982 - mean_squared_error: 0.0879 - acc: 0.8853 - val_loss: 0.3017 - val_mean_squared_error: 0.0911 - val_acc: 0.8746
Epoch 169/3000
 - 0s - loss: 0.2952 - mean_squared_error: 0.0872 - acc: 0.8911 - val_loss: 0.3015 - val_mean_squared_error: 0.0910 - val_acc: 0.8746
Epoch 170/3000
 - 0s - loss: 0.2967 - mean_squared_error: 0.0875 - acc: 0.8853 - val_loss: 0.3012 - val_mean_squared_error: 0.0910 - val_acc: 0.8746
Epoch 171/3000
 - 0s - loss: 0.2969 - mean_squared_error: 0.0880 - acc: 0.8864 - val_loss: 0.3009 - val_mean_squared_error: 0.0909 - val_acc: 0.8746
Epoch 172/3000
 - 0s - loss: 0.2963 - mean_squared_error: 0.0880 - acc: 0.8853 - val_loss: 0.3005 - val_mean_squared_error: 0.0908 - val_acc: 0.8746
Epoch 173/3000
 - 0s - loss: 0.2943 - mean_squared_error: 0.0870 - acc: 0.8841 - val_loss: 0.3001 - val_mean_squared_error: 0.0907 - val_acc: 0.8746
Epoch 174/3000
 - 0s - loss: 0.2949 - mean_squared_error: 0.0873 - acc: 0.8876 - val_loss: 0.2997 - val_me

Epoch 223/3000
 - 0s - loss: 0.2773 - mean_squared_error: 0.0820 - acc: 0.8946 - val_loss: 0.2906 - val_mean_squared_error: 0.0891 - val_acc: 0.8746
Epoch 224/3000
 - 0s - loss: 0.2762 - mean_squared_error: 0.0814 - acc: 0.8888 - val_loss: 0.2905 - val_mean_squared_error: 0.0890 - val_acc: 0.8746
Epoch 225/3000
 - 0s - loss: 0.2752 - mean_squared_error: 0.0816 - acc: 0.8876 - val_loss: 0.2905 - val_mean_squared_error: 0.0890 - val_acc: 0.8746
Epoch 226/3000
 - 0s - loss: 0.2765 - mean_squared_error: 0.0812 - acc: 0.8957 - val_loss: 0.2904 - val_mean_squared_error: 0.0890 - val_acc: 0.8746
Epoch 227/3000
 - 0s - loss: 0.2777 - mean_squared_error: 0.0820 - acc: 0.8922 - val_loss: 0.2904 - val_mean_squared_error: 0.0890 - val_acc: 0.8746
Epoch 228/3000
 - 0s - loss: 0.2770 - mean_squared_error: 0.0819 - acc: 0.8911 - val_loss: 0.2903 - val_mean_squared_error: 0.0890 - val_acc: 0.8711
Epoch 229/3000
 - 0s - loss: 0.2728 - mean_squared_error: 0.0802 - acc: 0.8934 - val_loss: 0.2902 - val_me

Epoch 278/3000
 - 0s - loss: 0.2609 - mean_squared_error: 0.0768 - acc: 0.8969 - val_loss: 0.2860 - val_mean_squared_error: 0.0884 - val_acc: 0.8676
Epoch 279/3000
 - 0s - loss: 0.2622 - mean_squared_error: 0.0771 - acc: 0.9003 - val_loss: 0.2859 - val_mean_squared_error: 0.0884 - val_acc: 0.8676
Epoch 280/3000
 - 0s - loss: 0.2581 - mean_squared_error: 0.0761 - acc: 0.8992 - val_loss: 0.2858 - val_mean_squared_error: 0.0884 - val_acc: 0.8676
Epoch 281/3000
 - 0s - loss: 0.2598 - mean_squared_error: 0.0761 - acc: 0.9038 - val_loss: 0.2857 - val_mean_squared_error: 0.0884 - val_acc: 0.8676
Epoch 282/3000
 - 0s - loss: 0.2596 - mean_squared_error: 0.0761 - acc: 0.9050 - val_loss: 0.2857 - val_mean_squared_error: 0.0884 - val_acc: 0.8676
Epoch 283/3000
 - 0s - loss: 0.2585 - mean_squared_error: 0.0762 - acc: 0.9015 - val_loss: 0.2856 - val_mean_squared_error: 0.0884 - val_acc: 0.8676
Epoch 284/3000
 - 0s - loss: 0.2588 - mean_squared_error: 0.0761 - acc: 0.8957 - val_loss: 0.2856 - val_me

Epoch 333/3000
 - 0s - loss: 0.2495 - mean_squared_error: 0.0735 - acc: 0.9050 - val_loss: 0.2843 - val_mean_squared_error: 0.0884 - val_acc: 0.8641
Epoch 334/3000
 - 0s - loss: 0.2459 - mean_squared_error: 0.0719 - acc: 0.9050 - val_loss: 0.2843 - val_mean_squared_error: 0.0884 - val_acc: 0.8641
Epoch 335/3000
 - 0s - loss: 0.2479 - mean_squared_error: 0.0726 - acc: 0.9050 - val_loss: 0.2842 - val_mean_squared_error: 0.0884 - val_acc: 0.8641
Epoch 336/3000
 - 0s - loss: 0.2503 - mean_squared_error: 0.0735 - acc: 0.9038 - val_loss: 0.2841 - val_mean_squared_error: 0.0884 - val_acc: 0.8641
Epoch 337/3000
 - 0s - loss: 0.2475 - mean_squared_error: 0.0729 - acc: 0.8992 - val_loss: 0.2841 - val_mean_squared_error: 0.0884 - val_acc: 0.8641
Epoch 338/3000
 - 0s - loss: 0.2484 - mean_squared_error: 0.0733 - acc: 0.9015 - val_loss: 0.2841 - val_mean_squared_error: 0.0884 - val_acc: 0.8641
Epoch 339/3000
 - 0s - loss: 0.2485 - mean_squared_error: 0.0727 - acc: 0.9096 - val_loss: 0.2840 - val_me

Epoch 388/3000
 - 0s - loss: 0.2374 - mean_squared_error: 0.0697 - acc: 0.9131 - val_loss: 0.2844 - val_mean_squared_error: 0.0887 - val_acc: 0.8606
Epoch 389/3000
 - 0s - loss: 0.2370 - mean_squared_error: 0.0692 - acc: 0.9085 - val_loss: 0.2844 - val_mean_squared_error: 0.0887 - val_acc: 0.8606
Epoch 390/3000
 - 0s - loss: 0.2361 - mean_squared_error: 0.0690 - acc: 0.9119 - val_loss: 0.2844 - val_mean_squared_error: 0.0887 - val_acc: 0.8606
Epoch 391/3000
 - 0s - loss: 0.2356 - mean_squared_error: 0.0690 - acc: 0.9119 - val_loss: 0.2844 - val_mean_squared_error: 0.0887 - val_acc: 0.8606
Epoch 392/3000
 - 0s - loss: 0.2343 - mean_squared_error: 0.0688 - acc: 0.9085 - val_loss: 0.2845 - val_mean_squared_error: 0.0887 - val_acc: 0.8606
Epoch 393/3000
 - 0s - loss: 0.2399 - mean_squared_error: 0.0707 - acc: 0.9085 - val_loss: 0.2845 - val_mean_squared_error: 0.0887 - val_acc: 0.8606
Epoch 394/3000
 - 0s - loss: 0.2348 - mean_squared_error: 0.0691 - acc: 0.9096 - val_loss: 0.2845 - val_me

(['loss', 'mean_squared_error', 'acc'],
 [0.28427964459312915, 0.08864035472948792, 0.8606271781157118])

In [24]:
nn_preds = model.predict_classes(x_test_norm)
nn_pos = model.predict_proba(x_test_norm)

In [25]:
accuracy(nn_preds), rmse(nn_pos), auroc(nn_pos), logloss(nn_pos)

(0.8606271777003485,
 0.29772530029229904,
 0.9460563077584354,
 0.28427964274522927)

In [ ]:
(0.8606271777003485,
 0.29772530029229904,
 0.9460563077584354,
 0.28427964274522927)

In [ ]:
train_df['gbc_preds'] = gbc.predict_proba(x_train).T[1]
test_df['gbc_preds'] = gbc.predict_proba(x_test).T[1]
train_df['rfc_preds'] = rfc.predict_proba(x_train).T[1]
test_df['rfc_preds'] = rfc.predict_proba(x_test).T[1]
train_df['etc_preds'] = etc.predict_proba(x_train).T[1]
test_df['etc_preds'] = etc.predict_proba(x_test).T[1]
train_df['nn_preds'] = model.predict_proba(x_train_norm)
test_df['nn_preds'] = model.predict_proba(x_test_norm)

In [ ]:
train_df.head()

In [ ]:
train_df.to_csv(f'{PATH}train_preds.csv')
test_df.to_csv(f'{PATH}test_preds.csv')